# Caderno para construção e teste do BOT

## Loading

In [1]:
## Loader do PDF
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader

path = "data\\manual_consolidado.pdf"

## Loaders structured and unstructured

#loader_un = UnstructuredPDFLoader(path)
loader_structured = PyMuPDFLoader(path)

pages_str = loader_structured.load()
#pages_str = loader_un.load()

## Cleaning

In [2]:
from langchain.text_splitter import CharacterTextSplitter

## Criando texto splitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=0
    )

## Split
texts = text_splitter.split_documents(pages_str)




In [3]:
### Testanto

texts[500].page_content

'O sistema gera uma confirmação da solicitação, clicar em “Ok”.\nO status final do processo fica como “Lote entregue”. Após a finalização, o processo não pode\nmais ser excluído e o botão “Excluir” fica indisponível.\nO novo lote pode ser consultado na aba “Lote fornecedor”, com uma nova sequência que\nfoi criada. O lote que deu origem ao processo de unitarização segue com o status\n“Enviado para unitarização”.\nConsultores:\nBruno Emanoel Calixto Ferreira\nUllysses Henrique Machado Ribas\nPágina:\n75 de 80'

In [4]:
## Função para remover espaço em branco

def remove_ws(d):
    text = d.page_content.replace("\n", "")
    d.page_content = text
    return d

# Clean texts by removing whitespace from each document
texts_cleaned = [remove_ws(d) for d in texts]

In [5]:
texts_cleaned[500].page_content

len(texts), len(texts_cleaned)

(5458, 5458)

## Retriever

In [ ]:
from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
import joblib

## Embeddings
embeddings = OllamaEmbeddings(
    model="llama3.2:1b")

# Create the vectorstore

vectorstore = InMemoryVectorStore.from_documents(
    documents=texts_cleaned,
    embedding=embeddings,
)
[]
# Use the vectorstore as a retriever
retriever = vectorstore.as_retriever()

# Salvar o retriever no arquivo
joblib.dump(retriever, 'data\\retriever.joblib')


In [ ]:
## Salvando retriever

import joblib

# Criar e salvar o retriever
retriever = ...  # O código que cria o retriever a partir do PDF dos manuais

# Salvar o retriever no arquivo


joblib.dump(retriever, 'data\\retriever.joblib')


In [ ]:
## Carregando retriever

import joblib

# Carregar o retriever salvo
retriever = joblib.load('data\\retriever.joblib')


## Chain and chats

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Função que o bot usará para responder a mensagens diretas

template = """
Você é um assistente especializado no sistema de gestão hospitalar Tasy, utilizado pela Fundação Hospitalar do Estado de Minas Gerais (FHEMIG).
Seu objetivo é ajudar os usuários a utilizarem o Tasy de forma eficiente, respondendo a perguntas com base nos manuais e documentos internos disponíveis.

Além disso, você utiliza a técnica de **Chain of Thought (CoT)** para resolver problemas mais complexos. Isso significa que você deve fornecer um raciocínio passo a passo para as questões que exigem várias etapas ou decisões. Explicite seu processo de raciocínio antes de chegar à resposta final.

### Instruções:

1. Sempre forneça uma resposta clara e objetiva com base nas informações extraídas dos manuais.
2. Utilize um raciocínio passo a passo (Chain of Thought) para resolver perguntas complexas ou processos que envolvem múltiplas etapas.
3. Se necessário, você pode buscar em múltiplos documentos para encontrar a melhor resposta.
4. Caso a resposta seja longa ou envolva várias etapas, quebre-a em passos claros e numerados.
5. Se o usuário pedir mais detalhes, você pode expandir a explicação ou oferecer links para seções específicas dos manuais.

### Estrutura de Resposta:
1. **Raciocínio Passo a Passo (Chain of Thought)**: Para perguntas que envolvem várias etapas ou decisões, forneça um raciocínio lógico que explique o processo antes de chegar à resposta final.
2. **Resumo Rápido**: Forneça uma visão geral da resposta em uma ou duas frases.
3. **Passos Detalhados** (se aplicável): Se o problema envolver etapas, forneça uma lista numerada.
4. **Referência ao Manual**: Sempre que possível, inclua referências ao manual, como o número da página ou seção relevante.

### Exemplos de Perguntas e Respostas:

Exemplo 1:
**Pergunta**: Como faço para gerar um relatório de pacientes?
**Resposta**:

1. **Chain of Thought**: Para gerar um relatório de pacientes no Tasy, precisamos entender qual módulo acessar, como configurar os filtros, e como gerar o relatório final. O sistema Tasy exige que o usuário escolha os filtros adequados para garantir que o relatório seja útil. Vamos passo a passo:
   - Primeiro, identifique o módulo correto (Relatórios).
   - Em seguida, selecione o tipo de relatório adequado (Relatório de Pacientes).
   - Depois, configure os filtros, como data, tipo de paciente, etc.
   - Finalmente, clique em "Gerar" para visualizar o relatório.

2. **Resumo Rápido**: Para gerar um relatório de pacientes, você deve acessar o módulo "Relatórios" e selecionar "Relatório de Pacientes".
3. **Passos Detalhados**: 
   - Vá para o menu principal.
   - Selecione "Relatórios".
   - Escolha "Relatório de Pacientes".
   - Defina os filtros necessários, como período ou tipo de paciente.
   - Clique em "Gerar".
4. **Referência ao Manual**: Consulte o manual do Tasy, página 235, para mais informações detalhadas sobre os filtros aplicáveis.

Exemplo 2:
**Pergunta**: Qual a função do campo "Status" no cadastro de pacientes?
**Resposta**:

1. **Chain of Thought**: O campo "Status" no cadastro de pacientes tem várias funções, incluindo o rastreamento da situação do paciente. Podemos começar compreendendo o que cada status representa. Por exemplo, um paciente ativo pode estar atualmente recebendo tratamento, enquanto um paciente inativo pode ter sido desligado do sistema. Vamos analisar como esse campo impacta a gestão de pacientes.
   - O campo "Status" pode ser configurado para refletir o estado do paciente.
   - Ele também pode ser utilizado por administradores para definir o acesso a determinados módulos do sistema.

2. **Resumo Rápido**: O campo "Status" indica a situação atual do paciente no sistema.
3. **Passos Detalhados**: Ele pode ser utilizado para identificar se o paciente está ativo, inativo ou internado. Pode ser atualizado pelo administrador.
4. **Referência ao Manual**: Ver manual do Tasy, seção 3.5, página 47.

{context}
"""

# Converter o template em prompt_template

prompt = ChatPromptTemplate.from_template(template)


## Configurar llm com Ollama

llm = ChatOllama(model="llama3.2:1b")

## Configurar chain

chain = chain = (
# The initial dictionary uses the retriever and user supplied query
    {"context":retriever,
     "query":RunnablePassthrough()}
# Feeds that context and query into the prompt then model & lastly 
# uses the ouput parser, do query for the data.
    |  prompt  | llm | StrOutputParser()
)



NameError: name 'retriever' is not defined

## Testando!

In [ ]:
chain.invoke("input")